In [12]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-07-23 22:20:04,310: Credentials are already in use. The existing account in the session will be replaced.


In [4]:
!pip install pennylane
import pennylane as qml

  Using cached PennyLane-0.16.0-py3-none-any.whl (514 kB)
  Using cached autoray-0.2.5-py3-none-any.whl (16 kB)
  Using cached autograd-1.3-py3-none-any.whl
  Using cached semantic_version-2.6.0-py3-none-any.whl (14 kB)
  Using cached future-0.18.2-py3-none-any.whl


In [5]:
!pip install pennylane-qiskit

  Using cached PennyLane_qiskit-0.16.0-py3-none-any.whl (21 kB)


In [13]:
from pennylane.optimize.gradient_descent import *
from pennylane import numpy as np

In [21]:
Optimizer = qml.QNGOptimizer(0.01, False, 0)

In [52]:
num_qubits = 4 #4 qubits
# device
#dev = qml.device('qiskit.ibmq', wires=num_qubits, backend='ibmq_qasm_simulator', provider=provider)
dev = qml.device("default.qubit", wires=4, shots=1024)
params = np.array([np.pi/2, np.pi/2, np.pi/2])

#user input for rotation gate angles
"""
params_1 = list(map(float, input().strip().split(' ')))
params_2 = list(map(float, input().strip().split(' ')))
params_3 = list(map(float, input().strip().split(' ')))
params_4 = list(map(float, input().strip().split(' ')))
params_5 = list(map(float, input().strip().split(' ')))
params_6 = list(map(float, input().strip().split(' ')))
params_list = list(map(float, input().strip().split(' ')))
"""

"""params_1 = default_params #[phi, theta, omega]
params_2 = default_params #[phi, theta, omega]
params_3 = default_params #[phi, theta, omega]
params_4 = default_params #[phi, theta, omega]
params_5 = default_params #[phi, theta, omega]
params_6 = default_params #[phi, theta, omega]
params_list = default_params #[phi, theta, omega]"""

@qml.qnode(dev)
def vqc(params, wires=4):
    #Cluster State
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.Hadamard(wires=3)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    qml.RY(np.pi, wires=0)
    qml.RY(np.pi, wires=1)
    qml.RY(np.pi, wires=2)
    qml.RY(np.pi, wires=3)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    #QConvLayer1
    qml.U3(params[0], params[1], params[2], wires=0)
    qml.U3(params[0], params[1], params[2], wires=1)
    qml.U3(params[0], params[1], params[2], wires=2)
    qml.U3(params[0], params[1], params[2], wires=3)
    qml.CRot(params[0], params[1], params[2], wires=[0,1])
    qml.CRot(params[0], params[1], params[2], wires=[2,3])
    #QConvLayer2
    qml.CRot(params_list[0], params_list[1], params_list[2], wires=[0,2])
    return qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(3))



In [27]:
help (qml.Hamiltonian)

Help on class Hamiltonian in module pennylane.vqe.vqe:

class Hamiltonian(builtins.object)
 |  Hamiltonian(coeffs, observables, simplify=False)
 |  
 |  Lightweight class for representing Hamiltonians for Variational Quantum
 |  Eigensolver problems.
 |  
 |  Hamiltonians can be expressed as linear combinations of observables, e.g.,
 |  :math:`\sum_{k=0}^{N-1} c_k O_k`.
 |  
 |  This class keeps track of the terms (coefficients and observables) separately.
 |  
 |  Args:
 |      coeffs (Iterable[float]): coefficients of the Hamiltonian expression
 |      observables (Iterable[Observable]): observables in the Hamiltonian expression
 |      simplify (bool): Specifies whether the Hamiltonian is simplified upon initialization
 |                       (like-terms are combined). The default value is `False`.
 |  
 |  .. seealso:: :class:`~.ExpvalCost`, :func:`~.molecular_hamiltonian`
 |  
 |  **Example:**
 |  
 |  A Hamiltonian can be created by simply passing the list of coefficients
 |  as

In [53]:
coeffs = [1,1,1]
obs = [qml.PauliZ(0), qml.PauliZ(0), qml.PauliZ(0)]
H = qml.Hamiltonian(coeffs, obs)

In [54]:
cost_fn = qml.ExpvalCost(vqc, H, dev)

In [55]:
%tb
theta_new = Optimizer.step(cost_fn, params)

QuantumFunctionError: A quantum function must return either a single measurement, or a nonempty sequence of measurements.

Traceback (most recent call last):
  File "<ipython-input-55-2de4377cbc25>", line 2, in <module>
    theta_new = Optimizer.step(cost_fn, params)
  File "/opt/conda/lib/python3.8/site-packages/pennylane/optimize/qng.py", line 215, in step
    x_out, _ = self.step_and_cost(
  File "/opt/conda/lib/python3.8/site-packages/pennylane/optimize/qng.py", line 194, in step_and_cost
    g, forward = self.compute_grad(qnode, (x,), dict())
  File "/opt/conda/lib/python3.8/site-packages/pennylane/optimize/gradient_descent.py", line 127, in compute_grad
    grad = g(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/pennylane/_grad.py", line 101, in __call__
    grad_value, ans = self._get_grad_fn(args)(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/autograd/wrap_util.py", line 20, in nary_f
    return unary_operator(unary_f, x, *nary_op_args, **nary_op_kwargs)
  File "/opt/conda/lib/python3.8/site-packages/pennylane/_grad.py", line 121, in _grad_with_forward
    raise T